# NE Climate Database
### MWWavelets
This file collects all the methods used in applying the wavelet transforms to the database created for the NE Climate wavelet project. It first constructs the NE Climate database, then defines all of the methods for applying the transforms, along with various other helper sections for creating noisy data and adding the COI's on plots. Statistical testing section is included, but is not applicable to the current data set.

In [2]:
import time
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Polygon

class StateID:
    """
    See above documentation for StateID class properties.
    """
    winter_months = [10, 11, 0, 1]
    spring_months = [2, 3, 4, 5]
    summer_months = [6, 7, 8, 9]
    
    def __init__(self, StateID, MaxTempDict, MinTempDict, PrecipDict):
        self._name = StateID
        self._max_temp = MaxTempDict
        self._min_temp = MinTempDict
        self._precip = PrecipDict
        
        # these will be set later with a different file/not used for anything but filtering stations
        self._coord = -9999 # tuple containing (latitude, longitude)
        self._elev = -9999
        
        self._max_temp_winter = {}
        self._max_temp_spring = {}
        self._max_temp_summer = {}
        self._min_temp_winter = {}
        self._min_temp_spring = {}
        self._min_temp_summer = {}
        self._precip_winter = {}
        self._precip_spring = {}
        self._precip_summer = {}
        
        self.make_winter_max_temp_dict()
        self.make_spring_max_temp_dict()
        self.make_summer_max_temp_dict()
        self.make_winter_min_temp_dict()
        self.make_spring_min_temp_dict()
        self.make_summer_min_temp_dict()
        self.make_winter_precip_dict()
        self.make_spring_precip_dict()
        self.make_summer_precip_dict()
    
    #getter functions
    def name(self):
        return self._name
    def max_temp(self):
        return self._max_temp
    def min_temp(self):
        return self._min_temp
    def precip(self):
        return self._precip
    
    #(lat, long) and elev
    def coord(self):
        return self._coord
    def elev(self):
        return self._elev
    
    def max_temp_winter(self):
        return self._max_temp_winter
    def max_temp_spring(self):
        return self._max_temp_spring
    def max_temp_summer(self):
        return self._max_temp_summer
    def min_temp_winter(self):
        return self._min_temp_winter
    def min_temp_spring(self):
        return self._min_temp_spring
    def min_temp_summer(self):
        return self._min_temp_summer
    def precip_winter(self):
        return self._precip_winter
    def precip_spring(self):
        return self._precip_spring
    def precip_summer(self):
        return self._precip_summer
    
    #seasonal dictionary builders
    def make_winter_max_temp_dict(self):
        new_dict = {}
        for year in self._max_temp.keys():
            new_dict[year] = self._max_temp.get(year)[0:2]
            new_dict[year] += self._max_temp.get(year)[10:12]
        self._max_temp_winter = new_dict
    def make_spring_max_temp_dict(self):
        new_dict = {}
        for year in self._max_temp.keys():
            new_dict[year] = self._max_temp.get(year)[2:6]
        self._max_temp_spring = new_dict
    def make_summer_max_temp_dict(self):
        new_dict = {}
        for year in self._max_temp.keys():
            new_dict[year] = self._max_temp.get(year)[6:10]
        self._max_temp_summer = new_dict
    def make_winter_min_temp_dict(self):
        new_dict = {}
        for year in self._min_temp.keys():
            new_dict[year] = self._min_temp.get(year)[0:2]
            new_dict[year] += self._min_temp.get(year)[10:12]
        self._min_temp_winter = new_dict
    def make_spring_min_temp_dict(self):
        new_dict = {}
        for year in self._min_temp.keys():
            new_dict[year] = self._min_temp.get(year)[2:6]
        self._min_temp_spring = new_dict
    def make_summer_min_temp_dict(self):
        new_dict = {}
        for year in self._min_temp.keys():
            new_dict[year] = self._min_temp.get(year)[6:10]
        self._min_temp_summer = new_dict
    def make_winter_precip_dict(self):
        new_dict = {}
        for year in self._precip.keys():
            new_dict[year] = self._precip.get(year)[0:2]
            new_dict[year] += self._precip.get(year)[10:12]
        self._precip_winter = new_dict
    def make_spring_precip_dict(self):
        new_dict = {}
        for year in self._precip.keys():
            new_dict[year] = self._precip.get(year)[2:6]
        self._precip_spring = new_dict
    def make_summer_precip_dict(self):
        new_dict = {}
        for year in self._precip.keys():
            new_dict[year] = self._precip.get(year)[6:10]
        self._precip_summer = new_dict
        
        
print("checkpoint 1")
with open('womack.csv') as csvfile:
    rows = csv.reader(csvfile, delimiter=',')
    all_data = [row for row in rows]
    
#removing the first row to leave just data
all_data = all_data[1:]
#making StateID, year and month values ints, z-scores floats
#
#NOTE: double lowers chance of roundoff error, but float uses less memory. currently using floats
#SEE: http://programmers.stackexchange.com/questions/188721/when-do-you-use-float-and-when-do-you-use-double
#
for i in all_data:
    i[0] = int(i[0])   #StateID
    i[1] = int(i[1])   #Year
    i[2] = int(i[2])   #Month
    i[3] = float(i[3]) #MaxTemp z-score
    i[4] = float(i[4]) #MinTemp z-score
    i[5] = float(i[5]) #Precip z-score

print("checkpoint 2")
#looping over the unique StateID list to create dictionaries and initialize objects
StateID_objects = []
for i in range(137):
    ID_max_temp_dict = {}
    ID_min_temp_dict = {}
    ID_precip_dict = {}
    for j in range(i*1380,(i+1)*1380):
        #we only check existence of key in the max_temp_dict because processing occurs
        #for all of the fields at once. max_temp_dict thus is a valid proxy for keys
        current_row = all_data[j]
        #cleaning the bad data
        #some stations were missing data, but no station missed more than 3 months of data
        #from January 1908 to September 2010 (Wilkie 24)
        #bad data entries were given values of -9999, replacing them with values of 0 (should not affect analysis)
        for k in range(3,6):
            if (current_row[k] == -9999.0):
                print("Cleaning at ID: ", str(current_row[0]))
                print("Year: "+str(current_row[1])+ "   Value: "+str(current_row[k]))
                current_row[k] = 0
                print("Set to: ", current_row[k])
        current_year = current_row[1]
        if current_year in ID_max_temp_dict:
            ID_max_temp_dict[current_year].append(current_row[3])
            ID_min_temp_dict[current_year].append(current_row[4])
            ID_precip_dict[current_year].append(current_row[5])
        else:
            ID_max_temp_dict[current_year] = [current_row[3]]
            ID_min_temp_dict[current_year] = [current_row[4]]
            ID_precip_dict[current_year] = [current_row[5]]
    current_State_ID = all_data[i*1380][0]
    StateID_objects.append(StateID(current_State_ID, ID_max_temp_dict, ID_min_temp_dict, ID_precip_dict))
    print("Completed Initilization of StateID Object: ", current_State_ID)

print("Completed Initilization of Objects!")

checkpoint 1
checkpoint 2
Completed Initilization of StateID Object:  62658
Completed Initilization of StateID Object:  63207
Completed Initilization of StateID Object:  67970
Completed Initilization of StateID Object:  68138
Completed Initilization of StateID Object:  170100
Completed Initilization of StateID Object:  170814
Completed Initilization of StateID Object:  171628
Completed Initilization of StateID Object:  172426
Completed Initilization of StateID Object:  172765
Completed Initilization of StateID Object:  173046
Cleaning at ID:  173944
Year: 1901   Value: -9999.0
Set to:  0
Cleaning at ID:  173944
Year: 1901   Value: -9999.0
Set to:  0
Completed Initilization of StateID Object:  173944
Completed Initilization of StateID Object:  174566
Completed Initilization of StateID Object:  175304
Completed Initilization of StateID Object:  176905
Cleaning at ID:  176937
Year: 1900   Value: -9999.0
Set to:  0
Cleaning at ID:  176937
Year: 1900   Value: -9999.0
Set to:  0
Cleaning at 

### Validation of StateID Class ###

In [3]:
# validating that objects contain correct indices
# checking to make sure that every year has 12 entries for every object

# this works as a valid check since we created the dictionaries using iteration, so any
# errors would be due to incorrect indexing and would show up as issues in the number of
# values assigned to one of the first or last years

def validate_StateID(StateID_object):
    start = StateID_object.max_temp().get(1900)
    end = StateID_object.max_temp().get(2014)
    if (len(start)!=12 or len(end)!=12):
        return False
    return True

def validate_all_StateID(StateID_object_list):
    for obj in StateID_objects:
        if validate_StateID(obj)==False:
            return "Error on StateID "+str(obj.name())+"!"
    return "All StateID objects are valid!"

validate_all_StateID(StateID_objects)

'All StateID objects are valid!'

In [4]:
#this is to check that the values match the csv file
for obj in StateID_objects:
    if obj.name() == 272999:
        print(obj.precip().get(1903))

[0.86237941, 2.117721569, 1.608463379, -1.711390114, 0, 0, 1.305917459, -0.785074336, -1.453419333, -0.524445127, -1.519106516, -0.242608153]


### Adding coordinates and elevation to Station ID Objects ###
Data obtained from http://www.surfacestations.org/ushcn_stationlist.htm, and stations missing from this data set were found at various sources related to https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/us-historical-climatology-network-ushcn.

Coordinates will be in a tuple, (latitude, longitude). Elevation is measured in units of feet.

In [5]:
with open('USHCN_ID_Collection_csv.csv') as csvfile:
    rows = csv.reader(csvfile, delimiter=',')
    all_data = [row for row in rows]

# removing the first row to leave just data
all_data = all_data[1:]
for i in all_data:
    i[0] = int(i[0])   #StateID
    i[1] = float(i[1])   #Latitude
    i[2] = float(i[2])   #Longitude
    i[3] = float(i[3])   #Elevation

# adding to StateID classes
for i in StateID_objects:
    for j in all_data:
        if i.name() == j[0]:
            i._coord = (j[1], j[2])
            i._elev = j[3]
            
print("Finished adding coordinates and elevation to StateIDs!")

Finished adding coordinates and elevation to StateIDs!


### Sorting to West/East of Appalachian Mountains ###
Coordinates of Appalachian mountains were chosen by using Google Maps topography map to track the eastmost "edge" of the mountain range. Six points were chose to approximate the extent of the mountain range. appalachian_coord takes the longitude (between -83.72 and -66.51 degrees) and returns the mountain range's corresponding latitude. We sort the StateID_objects into either West or East of the mountain range.

In [6]:
def appalachian_coord(longitude):
    """
    Input:
            longitude (float): the longitude coordinate
    Output:
            Returns the latitude (float) of the Appalachian mountain range East edge corresponding to the 
            given longitude coordinate.
    """
    mount_coords = [(34.50, -83.72), (37.58, -79.37), (40.00, -77.36), (41.01, -74.35), (43.80, -72.12), (47.82, -66.51)]
    if longitude < mount_coords[1][1]:
        b1 = mount_coords[0][0]
        slope1 = (mount_coords[1][0]-mount_coords[0][0])/(mount_coords[1][1]-mount_coords[0][1])
        return b1 + slope1*(longitude-mount_coords[0][1])
    elif longitude < mount_coords[2][1]:
        b2 = mount_coords[1][0]
        slope2 = (mount_coords[2][0]-mount_coords[1][0])/(mount_coords[2][1]-mount_coords[1][1])
        return b2 + slope2*(longitude-mount_coords[1][1])
    elif longitude < mount_coords[3][1]:
        b3 = mount_coords[2][0]
        slope3 = (mount_coords[3][0]-mount_coords[2][0])/(mount_coords[3][1]-mount_coords[2][1])
        return b3 + slope3*(longitude-mount_coords[2][1])
    elif longitude < mount_coords[4][1]:
        b4 = mount_coords[3][0]
        slope4 = (mount_coords[4][0]-mount_coords[3][0])/(mount_coords[4][1]-mount_coords[3][1])
        return b4 + slope4*(longitude-mount_coords[3][1])
    else:
        b5 = mount_coords[4][0]
        slope5 = (mount_coords[5][0]-mount_coords[4][0])/(mount_coords[5][1]-mount_coords[4][1])
        return b5 + slope5*(longitude-mount_coords[4][1])
    
def app_west_east(State_ID_object):
    """
    Input:
            State_ID_object (StateID_object): the StateID object to consider
    Output:
            True if the State_ID_object is located West of the Appalachian mountain range,
            False if the State_ID_object is East of the mountain range.
    """
    ID_coords = State_ID_object.coord()
    app_lat = appalachian_coord(ID_coords[1])
    if app_lat < ID_coords[0]:
        return True
    else:
        return False

def app_sorter(ID_obj_list):
    """
    Input:
            ID_obj_list (list of StateID_objects): the list of StateID objects to sort
    Output:
            A 2D list, first list is the StateID objects West of the Appalachian mountain range,
            second list is the StateID objects East of the mountain range.
    """
    west_list = []
    east_list = []
    for i in ID_obj_list:
        if app_west_east(i) == True:
            west_list.append(i)
        else:
            east_list.append(i)
    return [west_list, east_list]

Appalachian_StateID_objects = app_sorter(StateID_objects)
print("Number of StateIDs West of Appalachians: ", len(Appalachian_StateID_objects[0]))
print("Number of StateIDs East of Appalachians: ", len(Appalachian_StateID_objects[1]))

Number of StateIDs West of Appalachians:  83
Number of StateIDs East of Appalachians:  54


Processing and Displaying Data
------------------------------
1. The select_id function returns the object corresponding to the given ID.
2. The dict_to_list function takes the data dictionaries from the StateID classes and converts them to a list, with the values in chronological order.
3. The gaussian function is simply a function that gives the value of the normalized gaussian centered at mu for a specified x value and standard deviation. This is used when we want to use gaussian kernel smoothing.
4. The normal_kernel_constructor creates an array of uniformly sampled values of the gaussian function to smooth the data with.
5. The gaussian_smoother function takes an array of x-axis data, y-axis data, and a standard deviation, and smooths the y-axis data with a gaussian kernel according to the given standard deviation. It outputs a 2-D array containing the correct truncated x-axis data and smoothed y-axis data.

In [7]:
def select_id(id_number, id_obj_list):
    """
    Input:
            id_number (int): the ID number of the desired StateID.
            id_obj_list (list): the list of StateID objects created in the first part of the code.
    Output:
            the StateID object with name corresponding to the given id_number.
    """
    for obj in id_obj_list:
        if obj.name() == id_number:
            return obj
    
def dict_to_list(dictionary):
    converted_list = []
    #FOR SOME REASON 3.5 HAS THE KEYS IN A "VIEW" OBJECT, convert to list
    keys = list(dictionary.keys())
    keys.sort()
    for i in keys:
        converted_list += dictionary.get(i)
    return converted_list

def gaussian(x, mu, sig):
    return 1./(np.sqrt(2.*np.pi)*sig)*np.exp(-np.power((x - mu)/sig, 2.)/2)

def normal_kernel_constructor(std_dev):
    """
    Input:
            std_dev (float): the standard deviation of the kernel to be used, unit of years
    NOTE:   std_dev must give an int when multiplied by 12 so that we get an integer
            number of sampled points for the kernel!
    Output:
            list of discretely sampled kernel values corresponding to the normal
            distribution, covering 95% of kernel (2 sigma on either side of zero)
    """
    width = 2*std_dev #one sided width
    n_points = 2*width*12+1
    bins = n_points-1
    sampled_points = np.linspace(-width, width, num=n_points, endpoint=True)
    sampled_values = []
    for i in sampled_points:
        x = gaussian(i, 0, std_dev)*2*width/bins
        sampled_values.append(x)
    total = sum(sampled_values)
    renormalize_offset = (1-total)/n_points
    for i in range(len(sampled_values)):
        sampled_values[i] += renormalize_offset
    return sampled_values

def gaussian_smoother(x_array, y_array, std_dev):
    """
    Input:
            x_values (array_like): an array of points for the x-axis
            y_values (array_like): an array of the data to be smoothed
            std_dev (float):  the std_dev of the kernel to be used, unit of years. Refer to normal_kernel_constructor.
    Output:
            returns a 2-D numpy array. 1st array contains the x-values corresponding to the smoothed
            y-values. 2nd array contains the smoothed y-data.
    NOTE: Assumes that y_array, x_array are same dimension, and that the std_dev given does not make
          the gaussian kernel larger than the dimension of the y_array!
    """
    #NOTE: x-array is truncated at both endpoints since the gaussian window starts at the first
    #point where it does not extend past the data.
    n_of_width_points = int(2*std_dev*12) #number of points required on each side of mean for kernel
    x_array_dimension = len(x_array)
    truncated_x_list = list(x_array[n_of_width_points:x_array_dimension-n_of_width_points])
    
    kernel_values = normal_kernel_constructor(std_dev)
    smoothed_y_list = []
    for i in range(n_of_width_points, x_array_dimension-n_of_width_points):
        smooth_pass = 0
        for j in range(len(kernel_values)):
            smooth_pass += y_array[i+j-n_of_width_points]*kernel_values[j]
        smoothed_y_list.append(smooth_pass)
        
    return [truncated_x_list, smoothed_y_list]

#### Index and Averaging Methods ####

In [8]:
def nei_index(StateID_obj):
    """
    Returns an list containing the monthly NEI index values for the given StateID obj.
    Inputs:
            StateID_obj: a StateID object
    Outputs:
            Returns a 1-D list for the monthly NEI index for the given object.
    """
    max_temp_list = dict_to_list(StateID_obj.max_temp())
    min_temp_list = dict_to_list(StateID_obj.min_temp())
    precip_list = dict_to_list(StateID_obj.precip())
    nei_index_list = []
    for i in range(len(max_temp_list)):
        nei_value = (max_temp_list[i]+min_temp_list[i]+precip_list[i])/3
        nei_index_list.append(nei_value)
    return nei_index_list
    
def average_StateID_series(StateID_obj_list):
    """
    Returns a 4-D list containing the averaged max_temp, min_temp, precip and indexed values for each month
    across all of the objects in the list.
    Inputs:
            StateID_obj_list (list): list of StateID objects to average.
    Outputs:
            Returns a 4-D list. 
                average_StateID_series()[0] -> averaged max_temp list
                average_StateID_series()[1] -> averaged min_temp list
                average_StateID_series()[2] -> averaged precip list
                average_StateID_series()[3] -> averaged NEI index list
    """
    N = float(len(StateID_obj_list))
    max_temp_lists = []
    min_temp_lists = []
    precip_lists = []
    nei_index_lists = []
    for obj in StateID_obj_list:
        max_temp = dict_to_list(obj.max_temp())
        min_temp = dict_to_list(obj.min_temp())
        precip = dict_to_list(obj.precip())
        nei = nei_index(obj)
        max_temp_lists.append(max_temp)
        min_temp_lists.append(min_temp)
        precip_lists.append(precip)
        nei_index_lists.append(nei)
    avg_max_temp_list = list(np.divide(np.sum(max_temp_lists, axis=0), N))
    avg_min_temp_list = list(np.divide(np.sum(min_temp_lists, axis=0), N))
    avg_precip_list = list(np.divide(np.sum(precip_lists, axis=0), N))
    avg_nei_list = list(np.divide(np.sum(nei_index_lists, axis=0), N))
    return [avg_max_temp_list, avg_min_temp_list, avg_precip_list, avg_nei_list]

Generating Noisy Data
---------------------

In [9]:
def red_noise_generator(r, std_dev, length):
    """
    Using algorithm described by: http://www.atmos.washington.edu/~breth/classes/AM582/lect/lect8-notes.pdf
    NOTE: Additive red noise?
    
    Inputs:
            r (float): the lag-1 autocorrelation coefficient
            std_dev (float): the standard deviation for gaussian white noise used to generate the red noise sequence
            length (int): length of the sequence to be generated
            
    Output:
            list of floats for the red noise sequence
    """
    white_noise_sequence = np.random.normal(0, std_dev, length)
    white_noise_sequence = white_noise_sequence.tolist()
    red_noise_sequence = [white_noise_sequence[0]]
    for i in range(1, len(white_noise_sequence)):
        next_red_noise = r*red_noise_sequence[i-1] + np.sqrt(1-r*r)*white_noise_sequence[i]
        red_noise_sequence.append(next_red_noise)
    return red_noise_sequence

def line_generator(y_int, slope, x_list):
    """
    Generates a list of floats of a given length for a line. Used to make noisy data.
    Inputs:
            y_int (float): the y intercept of the line
            slope (float): the slope of the line
            x_list (list of floats): x values for the line. list is assumed to start at zero and be in ascending order.
    
    Output:
            list of y values (floats) corresponding to the desired line
    """
    y_list = [y_int]
    for i in range(len(x_list)-1):
        dy = (x_list[i+1]-x_list[i])*slope
        y_list.append(y_list[i]+dy)
    return y_list

def noisy_line_generator(x_list, y_list, noise_list):
    """
    Combines the given noise list with the given y_list to generate linear, noisy data.
    Inputs:
            x_list (list): the x values for the line, assumed to start at zero and be in ascending order
            y_list (list): the y values for the line
            noise_list (list): the noisy values to be combined with the linear data
            NOTE: All lists must be same length!
    
    Outputs:
            2-D list with first dimension containing the x values, second dimension containing the noisy y values
    """
    new_data = []
    for i in range(len(y_list)):
        new_data.append(y_list[i]+noise_list[i])
    return [x_list, new_data]

Wavelets
--------
Each different wavelet is its own class that encapsulates the following methods:
1. Scaling
2. Sampling
3. Renormalization

Each wavelet takes a float, defining the scale parameter of the wavelet, and calculates the number of sampled points allowed by this scale. Note that the number of sampled points will be odd, so that the central value has an equal number of points on either side for convolution. The wavelet is then uniformly sampled depending on the number of allowed sampling points. Since sampling provides only an approximation of the wavelet, we renormalize the sampled wavelet points so that the wavelet power is equal to one. This is done by numerically integrating the squared sampled wavelet points, subtracting the result from 1, averaging the difference over the number of points, and subtracting the averaged difference from each point. This is similar to renormalizing the sampled values of a gaussian PDF to preserve signal amplitude for smoothing purposes.

Implemented wavelets:
1. Morlet (complex)
2. Harr (real)

#### Morlet Wavelet ####
Defined in notes. Also have derivations of scaling methods and normalizing coefficients in notes.

In [10]:
class Morlet_Wavelet:  
    def __init__(self, scale, f):
        self._scale = scale
        self._f = f
    def f(self):
        return self._f
    def scale(self):
        return self._scale
    
    def morlet_function(self):
        """
        Calculates the values of the Morlet wavelet, centered at zero, w/ frequency f.
        Outputs:
                Returns the value of Morlet wavelet for each input value, list.
                NOTE: These values are complex numbers.
        """
        x = np.linspace(-2, 2, 2*self.sampling_points()+1)
        norm_coeff = np.power(np.pi, -1/4)
        complex_array = [-np.complex(0,1)*self.f()*i for i in x]
        neg_square_list = [-i*i/2 for i in x]
        sampled_values = [norm_coeff*np.exp(complex_array[i])*np.exp(neg_square_list[i]) for i in range(len(complex_array))]
        renorm_sampled_values = self.renormalize(sampled_values)
        final_sampled_values = [(1/np.sqrt(self._scale))*i for i in renorm_sampled_values]
        return final_sampled_values
    
    def scale_to_year(self):
        """
        Calculates the scale to year conversion for a given frequency and scale.
        Outputs:
                Returns the number of years the scale represents, float.
        """
        scale_factor = 4*np.pi/(self.f()+np.sqrt(2+self.f()*self.f()))
        return scale_factor*self.scale()
    
    def sampling_points(self):
        """
        Calculates the number of points to sample the wavelet with on one side (tail).
        Outputs:
                Returns the number of one sided sampling points, int.
        """
        years_for_scale = self.scale_to_year()
        one_sided_width = 2*years_for_scale # 2*scale corresponds to 95% of gaussian envelope [see notes]
        one_sided_months = 12*one_sided_width
        one_sided_points = np.ceil(one_sided_months)
        return one_sided_points
        
    def renormalize(self, sampled_values):
        """
        Renormalization helper function for morlet_function.
        Sets total power of sampled wavelet coefficients to 1.
        """
        # NOTE: It turns out that even for small scales the wavelet power is very close to 1
        # -> renormalization may not be necessary
        power = 0
        renorm_values = []
        d_power = 4/(2*self.sampling_points())
        for i in sampled_values:
            magnitude = np.absolute(i)
            power += magnitude*magnitude*d_power
        
        power_scale_factor = np.sqrt(1/power)
        
        for i in sampled_values:
            renorm_values.append(i*power_scale_factor)
        return renorm_values

#### Harr Wavelet ####
Scaling method and normalization are trivial. Scale directly corresponds to the year width of the wavelet. Notes on the Harr wavelet are found for 10/15.

The Harr wavelet corresponds to a difference in averages. The wavelet coefficient is the difference in the average of the first half of the data within the support of the wavelet, and the average of the second half of the data within the support. A positive value corresponds to the first half having a higher average, while a negative value corresponds to the second half having a higher average.

NOTE: I decided to implement the Harr wavelet so that the 1st value of the wavelet corresponds to the current value in the data set.

In [11]:
class Harr_Wavelet:
    """
    NOTE: SCALE MUST BE A MULTIPLE OF 1/6.
    """
    def __init__(self, scale):
        self._scale = scale
    def scale(self):
        return self._scale
    
    def harr_function(self):
        """
        Returns a list corresponding to the values of the Harr wavelet for each input value.
        These values are already normalized given the scale. Thus, each Harr wavelet class
        object corresponds to a daughter wavelet.
        Outputs:
                Returns the Harr wavelet values at each index, list.
        """
        scale = self.scale()
        normalized_value = 1/np.sqrt(12*scale)
        first_avg = [normalized_value]*(scale*6)
        second_avg = [-1*normalized_value]*(scale*6)
        harr_values = first_avg + second_avg
        return harr_values

Convolution Method
------------------
Takes a time series array and a wavelet array, returns the wavelet coefficients for the continuous wavelet transform.

Pads the array with zeros depending on COI of given wavelet array. Returned wavelet coefficient array entries correspond to the times in the original time series.

In [12]:
# these methods are for standardizing the Morlet wavelet scales
def scale_range(scale_min, scale_max):
    """
    Standardizes the granularity of scale ranges.
    """
    return list(np.arange(scale_min, scale_max+0.25, 0.25)) 

def scale_range_years(scale_list, morlet_freq):
    """
    Returns a list of the corresponding years in each scale for the Morlet wavelet with given frequency.
    """ 
    year_list = []
    scale_factor = 4*np.pi/(morlet_freq+np.sqrt(2+morlet_freq*morlet_freq))
    for i in scale_list:
        year_list.append(i*scale_factor)
    return year_list



# -------------------------------------------------------------------------------------------------------------- #
# these methods are only for the Morlet wavelet, different continuous convolution method for Harr wavelet
def continuous_convolution(time_series, wavelet_list):
    # this is convolving one Morlet wavelet with the time series
    """
    Inputs:
            time_series (array_like): time series values to be transformed
            wavelet_array (array_like): array of sampled, renormalized values from wavelet
            NOTE: wavelet_array must have an odd number of values! (for Morlet wavelet, may need different method
            for the Harr wavelet)
    Outputs:
            Returns an array with the Morlet wavelet coefficients.
    """
    coeffs_list = []
    n = len(time_series)
    m = len(wavelet_list)
    pad_len = int((m-1)/2) #one sided tail length
    padding_list = [0]*pad_len
    pad_time_series = padding_list + time_series + padding_list
    for i in range(pad_len, pad_len + n):
        coeff = 0
        for j in range(i-pad_len, i+pad_len+1):
            coeff += pad_time_series[j]*wavelet_list[j+pad_len-i]
        coeffs_list.append(coeff)
    return coeffs_list

def continuous_transform_morlet(time_series, scale_array, f):
    # this is convolving a set of Morlet wavelets with the time series
    """
    Inputs:
            time_series (array_like): time series values to be transformed
            scale_array (array_like): array of values for wavelet scales
            f (float): frequency of Morlet wavelet
    Outputs:
            2D array. Rows are wavelet coefficients for a given scale, columns are wavelet coefficient at each
            time series position.
    """
    wavelet_list = []
    wavelet_coefficients = []
    for i in scale_array:
        wvlt = Morlet_Wavelet(i, f)
        wavelet_list.append(wvlt.morlet_function())    
    for i in wavelet_list:
        wavelet_coefficients.append(continuous_convolution(time_series, i))
    return wavelet_coefficients



# these methods are for standardizing the granularity of the Harr wavelet scales
def harr_scale_range(scale_min, scale_max):
    """
    Standardizes the granularity of Harr scale ranges.
    Currently set so that each next scale corresponds to one more month of data added to each average.
    I.e. two data points are added onto the width of the next wavelet.
    """
    granularity = 1.0/6.0
    return list(np.arange(scale_min, scale_max+granularity, granularity))
# don't need to convert to years as scale = year for Harr wavelet



# -------------------------------------------------------------------------------------------------------------- #
# these methods are for convolution of the Harr wavelet
def harr_convolution(time_series, wavelet_list):
    # this is convolving one Harr wavelet with the time series
    """
    Inputs:
            time_series (array_like): time series values to be transformed
            wavelet_array (array_like): array of Harr wavelet values
            NOTE: wavelet_array must have an even number of values!
    Outputs:
            Returns an array with the Harr wavelet coefficients
    """
    coeffs_list = []
    n = len(time_series)
    m = len(wavelet_list)
    pad_len = m #we are only padding the end of the time series due to the Harr wavelet starting at current point
    padding_list = [0]*pad_len
    pad_time_series = time_series + padding_list
    for i in range(0, n):
        coeff = 0
        for j in range(i, i+m):
            coeff += pad_time_series[j]*wavelet_list[j-i]
        coeffs_list.append(coeff)
    return coeffs_list

def continuous_transform_harr(time_series, scale_array):
    # this is convolving a set of Harr wavelets with the time series
    """
    Inputs:
            time_series (array_like): time series values to be transformed
            scale_array (array_like): list of values for Harr wavelet scales
    Outputs:
            ND list. Rows are wavelet coefficients for each given scale, columns are wavelet coefficient at each
            time series position.
    """
    wavelet_list = []
    wavelet_coefficients = []
    for i in scale_array:
        wvlt = Harr_Wavelet(i)
        wavelet_list.append(wvlt.harr_function())    
    for i in wavelet_list:
        wavelet_coefficients.append(harr_convolution(time_series, i))
    return wavelet_coefficients
    
 

# -------------------------------------------------------------------------------------------------------------- #
# these methods are for computing the power of the wavelet coefficients
def complex_list_square(old_list):
    mag_list = np.absolute(old_list)
    return list(np.square(mag_list))

def wavelet_power_spectra(wavelet_coefficients):
    """
    Input:
            wavelet_coefficients (2D List): 2D list of complex wavelet coefficients. Rows are coefficients for each scale
    Output:
            2D list of the power of each wavelet coefficient. Rows are squared magnitudes at each scale
    """
    power_spectra_list = []
    for i in wavelet_coefficients:
        power_spectra_list.append(complex_list_square(i))
    return power_spectra_list

def normalized_wavelet_power_spectra(wavelet_spectra, variance):
    """
    Input:
            wavelet_spectra (2D List): the power of each wavelet coefficient. Rows are squared magnitudes at each scale
    Output:
            2D list of the normalized power of each wavelet coefficient. Rows are squared normalized magnitude at each scale
    """
    normalized_spectra = []
    for i in range(len(wavelet_spectra)):
        normalized_spectra_row = []
        for j in range(len(wavelet_spectra[0])):
            normalized_spectra_row.append((wavelet_spectra[i][j])/variance)
        normalized_spectra.append(normalized_spectra_row)
    return normalized_spectra

def wavelet_power_smoothing(x_data, wavelet_power_list, std_dev):
    """
    Input:
            x_data (list): the time coordinates of the data
            wavelet_power_list (list): the 2D list of wavelet power coefficients, rows correspond to the scale
            std_dev (float): the standard deviation of the gaussian kernel to smooth with, units of years
    Output:
            2D list, 1st entry is truncated x_data, 2nd entry is 2D list corresponding to the smoothed wavelet power
            spectra at each scale
    """
    smoothed_wavelet_power = []
    for scale_row in wavelet_power_list:
        new_data = gaussian_smoother(x_data, scale_row, std_dev)
        smoothed_wavelet_power.append(new_data[1])
    return [new_data[0], smoothed_wavelet_power]



# -------------------------------------------------------------------------------------------------------------- #
# these methods are for adding the COI to a plot
def coi_left(x_data_start, period_range):
    """
    Calculates the COI line for the left side of the COI of the Morlet wavelet.
    Inputs:
            x_data_start (float): the starting time coordinate of the time series
            period_range (list): the periods of the wavelets, in units of years
    Outputs:
            List of 2D coordinates for the polygon that corresponds to the COI
    """
    coi_x_data = [x_data_start]
    coi_x_data.append(2*period_range[0]+x_data_start)
    coi_x_data.append(2*period_range[-1]+x_data_start)
    coi_x_data.append(x_data_start)
    coi_y_data = [period_range[0], period_range[0], period_range[-1], period_range[-1]]
    coords = zip(coi_x_data, coi_y_data)
    coords_list = []
    for i in coords:
        coords_list.append(list(i))
    return coords_list

def coi_right(x_data_end, period_range):
    """
    Calculates the COI line for the left side of the COI of the Morlet wavelet.
    Inputs:
            x_data_start (float): the starting time coordinate of the time series
            period_range (list): the periods of the wavelets, in units of years
    Outputs:
            List of 2D coordinates for the polygon that corresponds to the COI
    """
    coi_x_data = [x_data_end, x_data_end, x_data_end-(2*period_range[-1]), x_data_end-(2*period_range[0])]
    coi_y_data = [period_range[0], period_range[-1], period_range[-1], period_range[0]]
    coords = zip(coi_x_data, coi_y_data)
    coords_list = []
    for i in coords:
        coords_list.append(list(i))
    return coords_list

### Statistical Testing ###

NOTE: This statistical testing algorithm has a problem where the amplitude of the wavelet coefficients is about an order of magnitude larger than expected. Do not know if this is due to the wavelet implementation, but currently the statistical testing algorithm implemented in Torrence and Compo does not work here.


UPDATE 3/12/17: The statistical testing fails not because the methods are written incorrectly, but because the data does not match the assumptions made by Torrence and Compo. In particular, they assume that the original data is well approximated by a red-noise lag-1 autocorrelated random process. In this case, because every value is the z-score of the original data taken with the mean of the entire data set, every value is highly correlated to every other value. This means that the data will not have a power spectrum close to that of red noise, and the statistical testing method fails.

In [13]:
# taken from: http://passel.unl.edu/pages/informationmodule.php?idinformationmodule=1130447119&topicorder=8&maxto=16&minto=1
chi_squared_two_95 = 5.99

def autocorrelation(n, time_series):
    """
    Calculates the autocorrelation at lag n. Assumes time_series length is long enough to use approximation of
    autocorrelation.
    Inputs:
            n (int): the lag value for the autocorrelation coefficient to be computed at
            time_series (list): the time series data
    Outputs:
            Returns the autocorrelation coefficient at lag n, float.
    """
    mean = sum(time_series)/float(len(time_series))
    numerator = 0
    denominator = 0
    for i in range(len(time_series)-n):
        numerator += (time_series[i]-mean)*(time_series[i+n]-mean)
    for i in range(len(time_series)):
        denominator += (time_series[i]-mean)*(time_series[i]-mean)
    return numerator/denominator

def sample_variance(time_series):
    """
    Calculates the unbiased sample variance of the time series.
    Inputs:
            time_series (list): the time series that we wish to find the variance of
    Outputs:
            Returns the unbiased sample variance of the time series, float.
    """
    mean = sum(time_series)/len(time_series)
    var_sum = 0
    for i in range(len(time_series)):
        var_sum += (time_series[i]-mean)*(time_series[i]-mean)
    return var_sum/(len(time_series)-1)

def theory_wavelet_spectra(alpha, period, N):
    """
    Calculates the theoretical normalized wavelet power for a wavelet of given Fourier period, assuming a red noise
    process of length N and lag-1 autocorrelation alpha.
    Inputs:
            alpha (float): the assumed lag-1 autocorrelation value of the time series
            period (float): the Fourier period of the wavelet of interest
                            NOTE: PERIOD UNIT MUST BE IN MONTHS!
            N (integer): the number of values in the time series
    Outputs:
            Returns the theoretical normalized wavelet power.
    """
    k = N/period
    P = (1-alpha*alpha)/(1+alpha*alpha-2*alpha*np.cos(2*np.pi*k/N))
    return P

def theory_95_confidence_spectra(alpha, period, N):
    """
    Same as theory_wavelet_spectra, but returns the theoretical 95% confidence wavelet power threshold.
    NOTE: For complex wavelet only, since complex wavelets have 2 DOF.
    """
    k = N/period
    P = (1-alpha*alpha)/(1+alpha*alpha-2*alpha*np.cos(2*np.pi*k/N))
    return (1/2)*P*chi_squared_two_95

def base_power_subtraction(wavelet_power_spectrum, year_range, alpha):
    """
    Removes the theoretical 95% confidence spectra from the wavelet power spectra. Anything below 0 is set to 0.
    The non-zero data thus represents the statistically significant peaks in the data.
    Inputs:
            wavelet_power_spectrum (2D list): the data containing the wavelet power for the time series. Rows should
                                              correspond to different scales
            year_range (list): the year values for each scale, used to calculate the theoretical spectra at each scale
            alpha (float): the lag-1 autocorrelation of the time series
    Outputs:
            2D List containing the theoretical 95% confidence spectra subtracted from the wavelet power spectra.
    NOTE: The year_range must match the scale range of wavelet_power_spectra!
    """
    subtracted_spectra = []
    N = len(wavelet_power_spectrum[0])
    theoretical_spectra = []
    for i in year_range:
        theoretical_spectra.append(theory_95_confidence_spectra(alpha, i*12, N))
    for i in range(len(wavelet_power_spectrum)):
        subtracted_year = []
        for j in range(len(wavelet_power_spectrum[0])):
            new_val = wavelet_power_spectrum[i][j]-theoretical_spectra[i]
            if (new_val < 0):
                new_val = 0
            subtracted_year.append(new_val)
        subtracted_spectra.append(subtracted_year)
    return subtracted_spectra

# Generating Batches of Random Data #
Creating batches of 100 sets of randomly generated time series to calculate approximate expected power distribution of wavelet coefficients for the raw data. The data will then be z-score transformed and the wavelet coefficients will be calculated for every set in the batch, and the average wavelet coefficients of the z-scored data will be compared to the average wavelet coefficients of the raw data to determine if there is a correlation between the two.

In [14]:
def no_baseline_red_noise_batch(r, std_dev, length, batch_size):
    """
    Creates a batch of red noise data with given length, standard deviation for Gaussian noise, and lag-1 autocorrelation r,
    that has no added baseline. 
    Inputs: 
        r (float): lag-1 autocorrelation for the red noise generator
        std_dev (float): standard deviation for Gaussian white-noise
        length (int): number of data points to generate
        batch_size (int): number of sets of data to generate
    Outputs:
        N-dim list where each list corresponds to one generated red noise data set of the given length.
    """
    batch_list = []
    for i in range(0, batch_size):
        batch_list.append(red_noise_generator(r, std_dev, length))
    return batch_list

# Transforming Batches of Random Data #
Applies transform to each set of the given batch. Separate function z-scores the sets in the batch, then applies transform. All functions return the AVERAGE wavelet coefficients for the whole batch.

In [15]:
def batch_wavelet_transform(batch_lists):
    """
    Takes a list of lists of raw data, returns the average wavelet coefficients for the data set.
    Inputs:
        batch_lists (list of lists): lists of raw data to be transformed
        NOTE: ASSUMES MORE THAN ONE LIST!
        NOTE: Assumes that each list is the same length.
    Outputs:
        List containing average wavelet coefficients.
    """
    wavelet_coeff_unavg = []
    batch_size = len(batch_lists)
    N = len(batch_lists[0])
    wavelet_scales = scale_range(0.5, 30)
    M = len(wavelet_scales)
    for i in range(0, batch_size):
        wavelet_coeff_unavg.append(continuous_transform_morlet(batch_lists[i], wavelet_scales, 2*np.pi))

    # now we average the coefficients. wavelet_coefficients_unaveraged is a list of 2-dim lists containing the wavelet
    # coefficients. 
    # creating a list of lists of zero with same dimension as the wavelet coefficients list of lists
    wavelet_coeff_avg = []
    zero_list = [0]*N
    for i in range(0, M):
        wavelet_coeff_avg.append(zero_list)
    # summing all wavelet values from each data set, averaging by the number of data sets
    for i in range(0, batch_size):
        for j in range(0, M):
            wavelet_coeff_avg[j] = [sum(x) for x in zip(wavelet_coeff_avg[j], wavelet_coeff_unavg[i][j])]
    for j in range(0, M):
        for k in range(0, N):
            wavelet_coeff_avg[j][k] = wavelet_coeff_avg[j][k]/float(batch_size)
    return wavelet_coeff_avg

def batch_z_score_transform(batch_lists):
    """
    Takes batch of data, returns the batch with each set as the z-scored data of the original set.
    Inputs:
        batch_lists (list of lists): the collection of data to be z-scored
    Outputs:
        A list of lists of the same dimension as the input, but each individual list is the z-scored list of the original data.
    """
    batch_size = len(batch_lists)
    z_score_lists = []
    for i in range(0, batch_size):
        z_score_set = z_score_list(batch_lists[i])
        z_score_lists.append(z_score_set)
    return z_score_lists

def batch_z_score_wavelet_transform(batch_lists):
    """
    Takes raw data, z-scores it, and then applies the Morlet wavelet transform to each set in the batch. For comparing to 
    the batch wavelet transforms.
    Inputs:
        batch_lists (list of lists): list of the raw data lists to be z-scored and transformed.
    Outputs:
        Average wavelet coefficients for the z-scored data.
    """
    z_scored_data = batch_z_score_transform(batch_lists)
    return batch_wavelet_transform(z_scored_data)
    
def z_score_list(data):
    """
    Returns the z-scored data.
    """
    mean = sum(data)/float(len(data))
    data_sum = 0
    for i in range(0, len(data)):
        data_sum += (data[i]-mean)*(data[i]-mean)
    std_dev = data_sum/float(len(data))
    data_z_score = []
    for i in range(0, len(data)):
        data_z_score.append((data[i]-mean)/std_dev)
    return data_z_score

CHANGELOG
---------

3/13/17 
- Split from NE Climate Database to condense methods w/o plotting scripts

3/29/17
- wrote lag-1 autocorrelated data batch generator
- wrote z_score_list method and batch_z_score_transform
- wrote batch_wavelet_transform

4/10/17
- adding lat/long/elev values to the StateIDs

4/18/17
- adding script to sort StateIDs by the Appalachian mountain range
- created sub-database with sorted StateIDs